In [29]:
import json
import pickle
import numpy as np
from operator import itemgetter
from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib

In [31]:
# Read files
with open('clean_data_leap/sorted_diag_list.pkl', 'rb') as f:
        sorted_diag_list = pickle.load(f)

with open('clean_data_leap/sorted_drug_list.pkl', 'rb') as f:
        sorted_drug_list = pickle.load(f)

with open('logistic_models_top538/matrix.pkl', 'rb') as f:
        feature_matrix = pickle.load(f)

with open('logistic_models_top538/test.pkl', 'rb') as f:
        test = pickle.load(f)

In [32]:
# Read models & generate probability matrix
prob_list = list()
for i in range(50):
    lr = joblib.load('logistic_models_top538/model' + str(i) + '.m')
    diag_matrix = np.matrix([feature_matrix[i] for i in test])
    diag_predict = lr.predict_proba(diag_matrix)
    prob = list()
    for predict in diag_predict:
        prob.append(predict[1])
    prob_list.append(prob)
prob_matrix = np.matrix(prob_list)

In [33]:
# Drug recommendation for test visits
mul_rec = []
with open('recommendation_leap/mul_rec_50.txt', "w") as f:
    prob_matrix_T = prob_matrix.T.tolist()
    for row in prob_matrix_T:
        drug_prob = {}
        for i in range(len(row)):
            drug_prob[i] = row[i]
        sorted_prob = sorted(drug_prob.items(), key=itemgetter(1), reverse=True)
        mul_rec.append(sorted_prob)
        for drug in sorted_prob:
            f.write("%s,%.4f;" %(sorted_drug_list[drug[0]][0], drug[1]))
        f.write('\n------------------------------------------------------\n')

In [34]:
# Save single disease drug recommendation dict
with open('recommendation_leap/mul_rec_50.pkl', 'wb') as f:
        pickle.dump(mul_rec, f)